In [349]:
# Load the required packages needed for performing data merging
%load_ext lab_black
import pandas as pd
import numpy as np

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [350]:
# Load shipment csv
ship = pd.read_csv(
    "C:/Users/Dean Huang/estimating-impact-of-opioids-2020-team6/00_source/SHIPMENT_agg.csv",
    sep=",",
)
# Check if there are anu duuplicate values
assert not ship.duplicated().any()

# Check if there are any null values
ship.isnull().any()

BUYER_STATE     False
BUYER_COUNTY    False
YEAR/MONTH      False
MME             False
dtype: bool

In [351]:
# Check the format of dataframe is correct
ship.head()

,BUYER_STATE,BUYER_COUNTY,YEAR/MONTH,MME
0,FL,ALACHUA,2006,82596.618688
1,FL,ALACHUA,2007,95259.627977
2,FL,ALACHUA,2008,114675.221922
3,FL,ALACHUA,2009,141281.048185
4,FL,ALACHUA,2010,150910.785876


In [352]:
# Create a new column that only has Year (for merging)
ship["Year"] = ship["YEAR/MONTH"].astype(str).str[:4].astype(int)
ship[ship["BUYER_STATE"] == "WA"]

,BUYER_STATE,BUYER_COUNTY,YEAR/MONTH,MME,Year
49987,WA,ADAMS,20061,211.94,2006
49988,WA,ADAMS,20062,202.97,2006
49989,WA,ADAMS,20063,175.37,2006
49990,WA,ADAMS,20064,213.54,2006
49991,WA,ADAMS,20065,187.66,2006
...,...,...,...,...,...
53258,WA,YAKIMA,201111,6265.72,2011
53259,WA,YAKIMA,201112,6589.18,2011
53260,WA,YAKIMA,201210,7325.65,2012
53261,WA,YAKIMA,201211,6702.34,2012


In [353]:
# Create a dictionary fpr mapping States to its abbreviation
States = {
    "FL": "Florida",
    "LA": "Louisiana",
    "MS": "Mississippi",
    "SC": "South Carolina",
    "WA": "Washington",
    "OR": "Oregon",
    "CO": "Colorado",
    "CA": "California",
    "TX": "Texas",
    "AR": "Arkansas",
    "NM": "New Mexico",
    "KS": "Kansas",
}
ship["FULL_STATES"] = ship["BUYER_STATE"].replace(States)
ship["FULL_STATES"].unique()

array(['Florida', 'Louisiana', 'Mississippi', 'South Carolina',
       'Arkansas', 'Kansas', 'New Mexico', 'Texas', 'California',
       'Colorado', 'Oregon', 'Washington'], dtype=object)

In [354]:
# Check if there are any null values
ship["FULL_STATES"].isnull().any()

False

In [355]:
# Load population 2000~2010 CSV
pop0 = pd.read_csv(
    "C:/Users/Dean Huang/estimating-impact-of-opioids-2020-team6/00_source/CensusEstimates2000-2010.csv",
    sep=",",
    encoding="ISO-8859-1",
)
pop0.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,ESTIMATESBASE2000,POPESTIMATE2000,POPESTIMATE2001,POPESTIMATE2002,POPESTIMATE2003,POPESTIMATE2004,POPESTIMATE2005,POPESTIMATE2006,POPESTIMATE2007,POPESTIMATE2008,POPESTIMATE2009,CENSUS2010POP,POPESTIMATE2010
0,40,3,6,1,0,Alabama,Alabama,4447207,4452173,4467634,4480089,4503491,4530729,4569805,4628981,4672840,4718206,4757938,4779736,4785298
1,50,3,6,1,1,Alabama,Autauga County,43751,44021,44889,45909,46800,48366,49676,51328,52405,53277,54135,54571,54632
2,50,3,6,1,3,Alabama,Baldwin County,140416,141342,144875,147957,151509,156266,162183,168121,172404,175827,179406,182265,183195
3,50,3,6,1,5,Alabama,Barbour County,29042,29015,28863,28653,28594,28287,28027,27861,27757,27808,27657,27457,27411
4,50,3,6,1,7,Alabama,Bibb County,19856,19913,21028,21199,21399,21721,22042,22099,22438,22705,22941,22915,22867


In [356]:
# Load population 2010~2019 CSV
pop1 = pd.read_csv(
    "C:/Users/Dean Huang/estimating-impact-of-opioids-2020-team6/00_source/CensusEstimates2010-2019 (Ship).csv",
    sep=",",
    encoding="ISO-8859-1",
)
pop1.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015,POPESTIMATE2016,POPESTIMATE2017,POPESTIMATE2018,POPESTIMATE2019
0,40,3,6,1,0,Alabama,Alabama,4779736,4780125,4785437,4799069,4815588,4830081,4841799,4852347,4863525,4874486,4887681,4903185
1,50,3,6,1,1,Alabama,Autauga County,54571,54597,54773,55227,54954,54727,54893,54864,55243,55390,55533,55869
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183112,186558,190145,194885,199183,202939,207601,212521,217855,223234
3,50,3,6,1,5,Alabama,Barbour County,27457,27455,27327,27341,27169,26937,26755,26283,25806,25157,24872,24686
4,50,3,6,1,7,Alabama,Bibb County,22915,22915,22870,22745,22667,22521,22553,22566,22586,22550,22367,22394


In [357]:
# Drop ESTIMATEBASE2000 because we are using CENSUS2000POP
pop0 = pop0.drop(["ESTIMATESBASE2000", "CENSUS2010POP", "POPESTIMATE2010"], axis=1)
pop0.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,POPESTIMATE2000,POPESTIMATE2001,POPESTIMATE2002,POPESTIMATE2003,POPESTIMATE2004,POPESTIMATE2005,POPESTIMATE2006,POPESTIMATE2007,POPESTIMATE2008,POPESTIMATE2009
0,40,3,6,1,0,Alabama,Alabama,4452173,4467634,4480089,4503491,4530729,4569805,4628981,4672840,4718206,4757938
1,50,3,6,1,1,Alabama,Autauga County,44021,44889,45909,46800,48366,49676,51328,52405,53277,54135
2,50,3,6,1,3,Alabama,Baldwin County,141342,144875,147957,151509,156266,162183,168121,172404,175827,179406
3,50,3,6,1,5,Alabama,Barbour County,29015,28863,28653,28594,28287,28027,27861,27757,27808,27657
4,50,3,6,1,7,Alabama,Bibb County,19913,21028,21199,21399,21721,22042,22099,22438,22705,22941


In [358]:
# Drop ESTIMATEBASE2010 because we are using CENSUS2010POP
pop1 = pop1.drop(["ESTIMATESBASE2010", "POPESTIMATE2010"], axis=1)
# Change CENSUS2010POP  to CENSUS2010
pop1.rename(columns={"CENSUS2010POP": "CENSUS2010"}, inplace=True)
pop1.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015,POPESTIMATE2016,POPESTIMATE2017,POPESTIMATE2018,POPESTIMATE2019
0,40,3,6,1,0,Alabama,Alabama,4779736,4799069,4815588,4830081,4841799,4852347,4863525,4874486,4887681,4903185
1,50,3,6,1,1,Alabama,Autauga County,54571,55227,54954,54727,54893,54864,55243,55390,55533,55869
2,50,3,6,1,3,Alabama,Baldwin County,182265,186558,190145,194885,199183,202939,207601,212521,217855,223234
3,50,3,6,1,5,Alabama,Barbour County,27457,27341,27169,26937,26755,26283,25806,25157,24872,24686
4,50,3,6,1,7,Alabama,Bibb County,22915,22745,22667,22521,22553,22566,22586,22550,22367,22394


In [359]:
# Subset only the states we are interested
States = [
    "Florida",
    "Louisiana",
    "Mississippi",
    "South Carolina",
    "Washington",
    "Oregon",
    "Colorado",
    "California",
    "Texas",
    "Arkansas",
    "New Mexico",
    "Kansas",
]
pop0_final = pop0[pop0["STNAME"].isin(States) == True]
pop1_final = pop1[pop1["STNAME"].isin(States) == True]

In [360]:
# Check the subset datafram only contain the states mentioned in the list
pop0_final["STNAME"].unique()

array(['Arkansas', 'California', 'Colorado', 'Florida', 'Kansas',
       'Louisiana', 'Mississippi', 'New Mexico', 'Oregon',
       'South Carolina', 'Texas', 'Washington'], dtype=object)

In [361]:
# Check the subset datafram only contain the states mentioned in the list
pop1_final["STNAME"].unique()

array(['Arkansas', 'California', 'Colorado', 'Florida', 'Kansas',
       'Louisiana', 'Mississippi', 'New Mexico', 'Oregon',
       'South Carolina', 'Texas', 'Washington'], dtype=object)

In [362]:
# Melt the dataframe so all the years is in one column
pop0_melt = pd.melt(
    pop0_final,
    ["SUMLEV", "REGION", "DIVISION", "STATE", "COUNTY", "STNAME", "CTYNAME"],
    var_name="Year",
    value_name="Population",
)
pop0_melt.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,Year,Population
0,40,3,7,5,0,Arkansas,Arkansas,POPESTIMATE2000,2678588
1,50,3,7,5,1,Arkansas,Arkansas County,POPESTIMATE2000,20776
2,50,3,7,5,3,Arkansas,Ashley County,POPESTIMATE2000,24179
3,50,3,7,5,5,Arkansas,Baxter County,POPESTIMATE2000,38446
4,50,3,7,5,7,Arkansas,Benton County,POPESTIMATE2000,154744


In [363]:
# Check population is correct after melting the dataframe
pop0_final[pop0_final["POPESTIMATE2000"] == 20776]

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,POPESTIMATE2000,POPESTIMATE2001,POPESTIMATE2002,POPESTIMATE2003,POPESTIMATE2004,POPESTIMATE2005,POPESTIMATE2006,POPESTIMATE2007,POPESTIMATE2008,POPESTIMATE2009
115,50,3,7,5,1,Arkansas,Arkansas County,20776,20593,20351,19981,19967,19954,19785,19434,19275,19134


In [364]:
# Melt the dataframe so all the years is in one column
pop1_melt = pd.melt(
    pop1_final,
    ["SUMLEV", "REGION", "DIVISION", "STATE", "COUNTY", "STNAME", "CTYNAME"],
    var_name="Year",
    value_name="Population",
)
pop1_melt.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,Year,Population
0,40,3,7,5,0,Arkansas,Arkansas,CENSUS2010,2915918
1,50,3,7,5,1,Arkansas,Arkansas County,CENSUS2010,19019
2,50,3,7,5,3,Arkansas,Ashley County,CENSUS2010,21853
3,50,3,7,5,5,Arkansas,Baxter County,CENSUS2010,41513
4,50,3,7,5,7,Arkansas,Benton County,CENSUS2010,221339


In [365]:
# Check population is correct after melting the dataframe
pop1_final[pop1_final["CENSUS2010"] == 19019.0]

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015,POPESTIMATE2016,POPESTIMATE2017,POPESTIMATE2018,POPESTIMATE2019
115,50,3,7,5,1,Arkansas,Arkansas County,19019,18871,18964,18755,18479,18330,18154,17872,17726,17486


In [366]:
# Concatenate these two dataframes into 1
popc = pd.concat([pop0_melt, pop1_melt], axis=0)
popc.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,Year,Population
0,40,3,7,5,0,Arkansas,Arkansas,POPESTIMATE2000,2678588
1,50,3,7,5,1,Arkansas,Arkansas County,POPESTIMATE2000,20776
2,50,3,7,5,3,Arkansas,Ashley County,POPESTIMATE2000,24179
3,50,3,7,5,5,Arkansas,Baxter County,POPESTIMATE2000,38446
4,50,3,7,5,7,Arkansas,Benton County,POPESTIMATE2000,154744


In [367]:
# Check everything is concatenate together
popc["Year"].unique()

array(['POPESTIMATE2000', 'POPESTIMATE2001', 'POPESTIMATE2002',
       'POPESTIMATE2003', 'POPESTIMATE2004', 'POPESTIMATE2005',
       'POPESTIMATE2006', 'POPESTIMATE2007', 'POPESTIMATE2008',
       'POPESTIMATE2009', 'CENSUS2010', 'POPESTIMATE2011',
       'POPESTIMATE2012', 'POPESTIMATE2013', 'POPESTIMATE2014',
       'POPESTIMATE2015', 'POPESTIMATE2016', 'POPESTIMATE2017',
       'POPESTIMATE2018', 'POPESTIMATE2019'], dtype=object)

In [368]:
# Remove county=0
popc = popc[popc["COUNTY"] != 0]
assert not (popc["COUNTY"] == 0).any()

In [369]:
# Remove the word 'County' & capitalize the name
# popc["CTYNAME"] = popc["CTYNAME"].str.replace("County", "").str.upper().replace(" ", "")
popc["CTYNAME"] = popc["CTYNAME"].str.rsplit(" ", 1)
popc["CTYNAME"] = popc["CTYNAME"].str[0]
popc["CTYNAME"] = popc["CTYNAME"].str.upper()
# popc["CTYNAME"] = popc["CTYNAME"].str.upper()
popc.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,Year,Population
1,50,3,7,5,1,Arkansas,ARKANSAS,POPESTIMATE2000,20776
2,50,3,7,5,3,Arkansas,ASHLEY,POPESTIMATE2000,24179
3,50,3,7,5,5,Arkansas,BAXTER,POPESTIMATE2000,38446
4,50,3,7,5,7,Arkansas,BENTON,POPESTIMATE2000,154744
5,50,3,7,5,9,Arkansas,BOONE,POPESTIMATE2000,34051


In [370]:
# Only keep the four digits for year
popc["Year"] = popc["Year"].str[-4:].astype(int)

In [371]:
popc.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,Year,Population
1,50,3,7,5,1,Arkansas,ARKANSAS,2000,20776
2,50,3,7,5,3,Arkansas,ASHLEY,2000,24179
3,50,3,7,5,5,Arkansas,BAXTER,2000,38446
4,50,3,7,5,7,Arkansas,BENTON,2000,154744
5,50,3,7,5,9,Arkansas,BOONE,2000,34051


In [372]:
# Remove unnecessary columns for population
drop = ["SUMLEV", "REGION", "DIVISION", "STATE", "COUNTY"]
popc1 = popc.drop(drop, axis=1)

In [373]:
popc["CTYNAME"].unique()

array(['ARKANSAS', 'ASHLEY', 'BAXTER', 'BENTON', 'BOONE', 'BRADLEY',
       'CALHOUN', 'CARROLL', 'CHICOT', 'CLARK', 'CLAY', 'CLEBURNE',
       'CLEVELAND', 'COLUMBIA', 'CONWAY', 'CRAIGHEAD', 'CRAWFORD',
       'CRITTENDEN', 'CROSS', 'DALLAS', 'DESHA', 'DREW', 'FAULKNER',
       'FRANKLIN', 'FULTON', 'GARLAND', 'GRANT', 'GREENE', 'HEMPSTEAD',
       'HOT SPRING', 'HOWARD', 'INDEPENDENCE', 'IZARD', 'JACKSON',
       'JEFFERSON', 'JOHNSON', 'LAFAYETTE', 'LAWRENCE', 'LEE', 'LINCOLN',
       'LITTLE RIVER', 'LOGAN', 'LONOKE', 'MADISON', 'MARION', 'MILLER',
       'MISSISSIPPI', 'MONROE', 'MONTGOMERY', 'NEVADA', 'NEWTON',
       'OUACHITA', 'PERRY', 'PHILLIPS', 'PIKE', 'POINSETT', 'POLK',
       'POPE', 'PRAIRIE', 'PULASKI', 'RANDOLPH', 'ST. FRANCIS', 'SALINE',
       'SCOTT', 'SEARCY', 'SEBASTIAN', 'SEVIER', 'SHARP', 'STONE',
       'UNION', 'VAN BUREN', 'WASHINGTON', 'WHITE', 'WOODRUFF', 'YELL',
       'ALAMEDA', 'ALPINE', 'AMADOR', 'BUTTE', 'CALAVERAS', 'COLUSA',
       'CONTRA COSTA', '

In [374]:
# Replace 'DEWITT' with 'DE WITT'
popc1["CTYNAME"].replace("DEWITT", "DE WITT", inplace=True)
popc1[popc1["CTYNAME"] == "DE WITT"]

,STNAME,CTYNAME,Year,Population
702,Texas,DE WITT,2000,19943
1637,Texas,DE WITT,2001,20066
2572,Texas,DE WITT,2002,20015
3507,Texas,DE WITT,2003,20034
4442,Texas,DE WITT,2004,20176
5377,Texas,DE WITT,2005,20342
6312,Texas,DE WITT,2006,20108
7247,Texas,DE WITT,2007,19961
8182,Texas,DE WITT,2008,19902
9117,Texas,DE WITT,2009,20048


In [375]:
# Replace 'Doña Ana" with "DONA ANA"
popc1["CTYNAME"].replace("DOÑA ANA", "DONA ANA", inplace=True)
popc1[popc1["CTYNAME"] == "DONA ANA"]

,STNAME,CTYNAME,Year,Population
530,New Mexico,DONA ANA,2000,175098
1465,New Mexico,DONA ANA,2001,176496
2400,New Mexico,DONA ANA,2002,178464
3335,New Mexico,DONA ANA,2003,182045
4270,New Mexico,DONA ANA,2004,184939
5205,New Mexico,DONA ANA,2005,189199
6140,New Mexico,DONA ANA,2006,193701
7075,New Mexico,DONA ANA,2007,197853
8010,New Mexico,DONA ANA,2008,200855
8945,New Mexico,DONA ANA,2009,205401


In [376]:
# Replace 'ST" with "SAINT"
popc1["CTYNAME"].replace("ST\.", "SAINT", regex=True, inplace=True)
popc1[popc1["CTYNAME"] == "SAINT FRANCIS"]

,STNAME,CTYNAME,Year,Population
62,Arkansas,SAINT FRANCIS,2000,29325
997,Arkansas,SAINT FRANCIS,2001,29072
1932,Arkansas,SAINT FRANCIS,2002,29023
2867,Arkansas,SAINT FRANCIS,2003,29170
3802,Arkansas,SAINT FRANCIS,2004,28905
4737,Arkansas,SAINT FRANCIS,2005,28639
5672,Arkansas,SAINT FRANCIS,2006,28496
6607,Arkansas,SAINT FRANCIS,2007,28394
7542,Arkansas,SAINT FRANCIS,2008,28110
8477,Arkansas,SAINT FRANCIS,2009,28336


In [377]:
# Replace 'SAINT JOHN THE BAPTIST" with "ST JOHN THE BAPTIST"
popc1["CTYNAME"].replace("SAINT JOHN THE BAPTIST", "ST JOHN THE BAPTIST", inplace=True)
popc1[popc1["CTYNAME"] == "ST JOHN THE BAPTIST"]

,STNAME,CTYNAME,Year,Population
422,Louisiana,ST JOHN THE BAPTIST,2000,43248
1357,Louisiana,ST JOHN THE BAPTIST,2001,43580
2292,Louisiana,ST JOHN THE BAPTIST,2002,43878
3227,Louisiana,ST JOHN THE BAPTIST,2003,44316
4162,Louisiana,ST JOHN THE BAPTIST,2004,44822
5097,Louisiana,ST JOHN THE BAPTIST,2005,45296
6032,Louisiana,ST JOHN THE BAPTIST,2006,47296
6967,Louisiana,ST JOHN THE BAPTIST,2007,47386
7902,Louisiana,ST JOHN THE BAPTIST,2008,46811
8837,Louisiana,ST JOHN THE BAPTIST,2009,46336


In [378]:
# Replace 'LASALLE" with "LA SALLE"
popc1["CTYNAME"].replace("LASALLE", "LA SALLE", inplace=True)
popc1[popc1["CTYNAME"] == "LA SALLE"]

,STNAME,CTYNAME,Year,Population
404,Louisiana,LA SALLE,2000,14271
782,Texas,LA SALLE,2000,5898
1339,Louisiana,LA SALLE,2001,14207
1717,Texas,LA SALLE,2001,5934
2274,Louisiana,LA SALLE,2002,14359
2652,Texas,LA SALLE,2002,6098
3209,Louisiana,LA SALLE,2003,14356
3587,Texas,LA SALLE,2003,6169
4144,Louisiana,LA SALLE,2004,14366
4522,Texas,LA SALLE,2004,6322


In [379]:
# Replace 'DESOTO" with "DE SOTO"
popc1["CTYNAME"].replace("DESOTO", "DE SOTO", inplace=True)
popc1[popc1["CTYNAME"] == "DE SOTO"]

,STNAME,CTYNAME,Year,Population
213,Florida,DE SOTO,2000,32196
390,Louisiana,DE SOTO,2000,25502
456,Mississippi,DE SOTO,2000,108714
1148,Florida,DE SOTO,2001,32402
1325,Louisiana,DE SOTO,2001,25449
1391,Mississippi,DE SOTO,2001,113401
2083,Florida,DE SOTO,2002,32446
2260,Louisiana,DE SOTO,2002,25598
2326,Mississippi,DE SOTO,2002,118603
3018,Florida,DE SOTO,2003,33271


In [380]:
# Replace 'DESOTO" with "DE SOTO"
ship["BUYER_COUNTY"].replace("DESOTO", "DE SOTO", inplace=True)
ship[ship["BUYER_COUNTY"] == "DE SOTO"]

,BUYER_STATE,BUYER_COUNTY,YEAR/MONTH,MME,Year,FULL_STATES
84,FL,DE SOTO,2006,7179.580012,2006,Florida
85,FL,DE SOTO,2007,7726.090772,2007,Florida
86,FL,DE SOTO,2008,7953.604365,2008,Florida
87,FL,DE SOTO,2009,9826.920240,2009,Florida
88,FL,DE SOTO,2010,13087.621238,2010,Florida
89,FL,DE SOTO,2011,18280.029405,2011,Florida
90,FL,DE SOTO,2012,16695.973793,2012,Florida
566,LA,DE SOTO,2006,3884.347863,2006,Louisiana
567,LA,DE SOTO,2007,4720.608166,2007,Louisiana
568,LA,DE SOTO,2008,5600.356138,2008,Louisiana


In [381]:
popc1.head()

,STNAME,CTYNAME,Year,Population
1,Arkansas,ARKANSAS,2000,20776
2,Arkansas,ASHLEY,2000,24179
3,Arkansas,BAXTER,2000,38446
4,Arkansas,BENTON,2000,154744
5,Arkansas,BOONE,2000,34051


In [382]:
popc2 = popc1[popc1.Year.isin([2006, 2007, 2008, 2009.2010, 2011, 2012])]

In [383]:
popc2.head()

,STNAME,CTYNAME,Year,Population
5611,Arkansas,ARKANSAS,2006,19785
5612,Arkansas,ASHLEY,2006,22501
5613,Arkansas,BAXTER,2006,40867
5614,Arkansas,BENTON,2006,195569
5615,Arkansas,BOONE,2006,36056


In [384]:
# Left Join Shipment & Population
merge = pd.merge(
    ship,
    popc1,
    how="left",
    left_on=["BUYER_COUNTY", "Year", "FULL_STATES"],
    right_on=["CTYNAME", "Year", "STNAME"],
)

In [385]:
merge.head()

,BUYER_STATE,BUYER_COUNTY,YEAR/MONTH,MME,Year,FULL_STATES,STNAME,CTYNAME,Population
0,FL,ALACHUA,2006,82596.618688,2006,Florida,Florida,ALACHUA,239506
1,FL,ALACHUA,2007,95259.627977,2007,Florida,Florida,ALACHUA,242685
2,FL,ALACHUA,2008,114675.221922,2008,Florida,Florida,ALACHUA,244888
3,FL,ALACHUA,2009,141281.048185,2009,Florida,Florida,ALACHUA,246657
4,FL,ALACHUA,2010,150910.785876,2010,Florida,Florida,ALACHUA,247336


In [386]:
# Look for null values
merge[merge["CTYNAME"].isnull()]

,BUYER_STATE,BUYER_COUNTY,YEAR/MONTH,MME,Year,FULL_STATES,STNAME,CTYNAME,Population


In [387]:
# Drop duplicated Columns
merge = merge.drop(["CTYNAME", "STNAME"], axis=1)

In [388]:
merge.head()

,BUYER_STATE,BUYER_COUNTY,YEAR/MONTH,MME,Year,FULL_STATES,Population
0,FL,ALACHUA,2006,82596.618688,2006,Florida,239506
1,FL,ALACHUA,2007,95259.627977,2007,Florida,242685
2,FL,ALACHUA,2008,114675.221922,2008,Florida,244888
3,FL,ALACHUA,2009,141281.048185,2009,Florida,246657
4,FL,ALACHUA,2010,150910.785876,2010,Florida,247336


In [389]:
# Create a column call MME/CAP
merge["MME/CAP"] = round(merge["MME"] / merge["Population"], 4)
merge.head()

,BUYER_STATE,BUYER_COUNTY,YEAR/MONTH,MME,Year,FULL_STATES,Population,MME/CAP
0,FL,ALACHUA,2006,82596.618688,2006,Florida,239506,0.3449
1,FL,ALACHUA,2007,95259.627977,2007,Florida,242685,0.3925
2,FL,ALACHUA,2008,114675.221922,2008,Florida,244888,0.4683
3,FL,ALACHUA,2009,141281.048185,2009,Florida,246657,0.5728
4,FL,ALACHUA,2010,150910.785876,2010,Florida,247336,0.6101


In [390]:
# Group by state and county then count the number of values
merge["checker"] = merge.groupby(["BUYER_STATE", "BUYER_COUNTY"], as_index=False)[
    ["YEAR/MONTH"]
].transform("count")

In [391]:
merge.head()

,BUYER_STATE,BUYER_COUNTY,YEAR/MONTH,MME,Year,FULL_STATES,Population,MME/CAP,checker
0,FL,ALACHUA,2006,82596.618688,2006,Florida,239506,0.3449,7
1,FL,ALACHUA,2007,95259.627977,2007,Florida,242685,0.3925,7
2,FL,ALACHUA,2008,114675.221922,2008,Florida,244888,0.4683,7
3,FL,ALACHUA,2009,141281.048185,2009,Florida,246657,0.5728,7
4,FL,ALACHUA,2010,150910.785876,2010,Florida,247336,0.6101,7


In [392]:
# Dop unnecessary columns
merge1 = merge.drop(["Year", "MME", "Population"], axis=1)

In [393]:
# For FL, LA, MS, SC the count should be equal to 7
# Subset dataframe that has incomplete values
missing1 = merge1[
    (merge1["checker"] != 7) & (merge1.BUYER_STATE.isin(["FL", "LA", "MS", "SC"]))
]
missing1.shape

(6, 6)

In [394]:
missing1

,BUYER_STATE,BUYER_COUNTY,YEAR/MONTH,FULL_STATES,MME/CAP,checker
140,FL,GLADES,2009,Florida,0.2046,4
141,FL,GLADES,2010,Florida,0.5037,4
142,FL,GLADES,2011,Florida,0.3620,4
143,FL,GLADES,2012,Florida,0.1498,4
543,LA,CAMERON,2011,Louisiana,0.0003,2
544,LA,CAMERON,2012,Louisiana,0.0043,2


In [395]:
# Removed the 6 "incomplete" rows
merge_clean = merge1[
    ~((merge1["checker"] != 7) & (merge1.BUYER_STATE.isin(["FL", "LA", "MS", "SC"])))
]
merge_clean.shape

(53257, 6)

In [396]:
# For the rest of states the count should be equal to 84
# Subset dataframe that has incomplete values
missing2 = merge_clean[
    (merge_clean["checker"] != 84)
    & (~merge_clean.BUYER_STATE.isin(["FL", "LA", "MS", "SC"]))
]
missing2.shape

(1989, 6)

In [397]:
# Removed the 1989 "incomplete" rows
merge_clean2 = merge_clean[
    ~(
        (merge_clean["checker"] != 84)
        & (~merge_clean.BUYER_STATE.isin(["FL", "LA", "MS", "SC"]))
    )
]
merge_clean2.shape

(51268, 6)

In [398]:
missing1

,BUYER_STATE,BUYER_COUNTY,YEAR/MONTH,FULL_STATES,MME/CAP,checker
140,FL,GLADES,2009,Florida,0.2046,4
141,FL,GLADES,2010,Florida,0.5037,4
142,FL,GLADES,2011,Florida,0.3620,4
143,FL,GLADES,2012,Florida,0.1498,4
543,LA,CAMERON,2011,Louisiana,0.0003,2
544,LA,CAMERON,2012,Louisiana,0.0043,2


In [399]:
# Calculate the years that are not present
Complete = [2006, 2007, 2008, 2009, 2010, 2011, 2012]
Missing = {}
for i in missing1["BUYER_COUNTY"].unique():
    present = []
    for j in range(len(missing1.index)):
        if missing1.iloc[j, 1] == i:
            present.append(missing1.iloc[j, 2])
    mis = np.setdiff1d(Complete, present).tolist()
    Missing.update({i: mis})

In [400]:
# Check the accuracy
Missing

{'GLADES': [2006, 2007, 2008], 'CAMERON': [2006, 2007, 2008, 2009, 2010]}

In [401]:
# Create empty rows for the missing years
for key, value in Missing.items():
    for year in value:
        df = pd.DataFrame(
            [[np.nan, key, year, np.nan, 0, np.nan]],
            columns=[
                "BUYER_STATE",
                "BUYER_COUNTY",
                "YEAR/MONTH",
                "FULL_STATES",
                "MME/CAP",
                "checker",
            ],
        )
        missing1 = pd.concat([missing1, df], ignore_index=False)

In [402]:
missing1

,BUYER_STATE,BUYER_COUNTY,YEAR/MONTH,FULL_STATES,MME/CAP,checker
140,FL,GLADES,2009,Florida,0.2046,4.0
141,FL,GLADES,2010,Florida,0.5037,4.0
142,FL,GLADES,2011,Florida,0.3620,4.0
143,FL,GLADES,2012,Florida,0.1498,4.0
543,LA,CAMERON,2011,Louisiana,0.0003,2.0
544,LA,CAMERON,2012,Louisiana,0.0043,2.0
0,NaN,GLADES,2006,NaN,0.0000,NaN
0,NaN,GLADES,2007,NaN,0.0000,NaN
0,NaN,GLADES,2008,NaN,0.0000,NaN
0,NaN,CAMERON,2006,NaN,0.0000,NaN


In [403]:
# Fill Missing States
missing1["BUYER_STATE"] = missing1.groupby("BUYER_COUNTY")["BUYER_STATE"].transform(
    lambda v: v.fillna(method="ffill")
)

In [404]:
# Fill Missing Full States
missing1["FULL_STATES"] = missing1.groupby("BUYER_COUNTY")["FULL_STATES"].transform(
    lambda v: v.fillna(method="ffill")
)

In [405]:
# Fill Missing Full Checker
missing1["checker"] = missing1.groupby("BUYER_COUNTY")["checker"].transform(
    lambda v: v.fillna(method="ffill")
)

In [406]:
dates = []
for i in merge_clean[
    (merge_clean["BUYER_COUNTY"] == "ADAMS") & (merge_clean["BUYER_STATE"] == "WA")
]["YEAR/MONTH"]:
    dates.append(i)

In [407]:
dates

[20061,
 20062,
 20063,
 20064,
 20065,
 20066,
 20067,
 20068,
 20069,
 20071,
 20072,
 20073,
 20074,
 20075,
 20076,
 20077,
 20078,
 20079,
 20081,
 20082,
 20083,
 20084,
 20085,
 20086,
 20087,
 20088,
 20089,
 20091,
 20092,
 20093,
 20094,
 20095,
 20096,
 20097,
 20098,
 20099,
 20101,
 20102,
 20103,
 20104,
 20105,
 20106,
 20107,
 20108,
 20109,
 20111,
 20112,
 20113,
 20114,
 20115,
 20116,
 20117,
 20118,
 20119,
 20121,
 20122,
 20123,
 20124,
 20125,
 20126,
 20127,
 20128,
 20129,
 200610,
 200611,
 200612,
 200710,
 200711,
 200712,
 200810,
 200811,
 200812,
 200910,
 200911,
 200912,
 201010,
 201011,
 201012,
 201110,
 201111,
 201112,
 201210,
 201211,
 201212]

In [408]:
missing2.head()

,BUYER_STATE,BUYER_COUNTY,YEAR/MONTH,FULL_STATES,MME/CAP,checker
2806,AR,CLEVELAND,20061,Arkansas,0.0012,34
2807,AR,CLEVELAND,20062,Arkansas,0.0015,34
2808,AR,CLEVELAND,20063,Arkansas,0.0014,34
2809,AR,CLEVELAND,20064,Arkansas,0.0004,34
2810,AR,CLEVELAND,20065,Arkansas,0.0012,34


In [409]:
# Calculate the months that are not present
Complete2 = dates
Missing2 = {}
for i in missing2["BUYER_COUNTY"].unique():
    present = []
    for j in range(len(missing2.index)):
        if missing2.iloc[j, 1] == i:
            present.append(missing2.iloc[j, 2])
    mis = np.setdiff1d(Complete2, present).tolist()
    Missing2.update({i: mis})

In [410]:
Missing2

{'CLEVELAND': [20091,
  20092,
  20093,
  20094,
  20095,
  20096,
  20097,
  20098,
  20099,
  20101,
  20102,
  20103,
  20104,
  20105,
  20106,
  20107,
  20108,
  20109,
  20111,
  20112,
  20113,
  20114,
  20115,
  20116,
  20117,
  20118,
  20119,
  20121,
  20122,
  20123,
  20124,
  20125,
  20126,
  20127,
  20128,
  20129,
  200811,
  200812,
  200910,
  200911,
  200912,
  201010,
  201011,
  201012,
  201110,
  201111,
  201112,
  201210,
  201211,
  201212],
 'KIOWA': [20076,
  20077,
  20078,
  20079,
  20081,
  20082,
  20083,
  20084,
  20085,
  20086,
  20087,
  20088,
  20089,
  20091,
  20092,
  20093,
  20094,
  20095,
  20096,
  20097,
  20098,
  20099,
  20101,
  20102,
  20103,
  20104,
  20105,
  20106,
  20107,
  20108,
  20109,
  20111,
  20112,
  20113,
  20114,
  20115,
  20116,
  20117,
  20118,
  20119,
  200710,
  200711,
  200712,
  200810,
  200811,
  200812,
  200910,
  200911,
  200912,
  201010,
  201011,
  201012,
  201110,
  201112],
 'LANE': [20

In [411]:
# Create empty rows for the missing months
for key, value in Missing2.items():
    for year in value:
        df = pd.DataFrame(
            [[np.nan, key, year, np.nan, 0, np.nan]],
            columns=[
                "BUYER_STATE",
                "BUYER_COUNTY",
                "YEAR/MONTH",
                "FULL_STATES",
                "MME/CAP",
                "checker",
            ],
        )
        missing2 = pd.concat([missing2, df], ignore_index=False)

In [412]:
# Fill Missing States
missing2["BUYER_STATE"] = missing2.groupby("BUYER_COUNTY")["BUYER_STATE"].transform(
    lambda v: v.fillna(method="ffill")
)

In [413]:
# Fill Missing Full States
missing2["FULL_STATES"] = missing2.groupby("BUYER_COUNTY")["FULL_STATES"].transform(
    lambda v: v.fillna(method="ffill")
)

In [414]:
# Fill Missing Checker
missing2["checker"] = missing2.groupby("BUYER_COUNTY")["checker"].transform(
    lambda v: v.fillna(method="ffill")
)

In [415]:
missing2.tail()

,BUYER_STATE,BUYER_COUNTY,YEAR/MONTH,FULL_STATES,MME/CAP,checker
0,OR,WHEELER,201110,Oregon,0.0,23.0
0,OR,WHEELER,201111,Oregon,0.0,23.0
0,OR,WHEELER,201112,Oregon,0.0,23.0
0,OR,WHEELER,201211,Oregon,0.0,23.0
0,OR,WHEELER,201212,Oregon,0.0,23.0


In [416]:
# Concatenate all dataframes together
merge_final = pd.concat([merge_clean2, missing1], ignore_index=False)
merge_final = pd.concat([merge_final, missing2], ignore_index=False)

In [417]:
# Group by state and county then count the number of values again
merge_final["final_checker"] = merge_final.groupby(
    ["BUYER_STATE", "BUYER_COUNTY"], as_index=False
)[["YEAR/MONTH"]].transform("count")

In [418]:
# Check if there are any missing rows again
merge_final[
    (merge_final["final_checker"] != 7)
    & (merge_final.BUYER_STATE.isin(["FL", "LA", "MS", "SC"]))
]

,BUYER_STATE,BUYER_COUNTY,YEAR/MONTH,FULL_STATES,MME/CAP,checker,final_checker


In [419]:
# For the rest of states the count should be equal to 84
# Subset dataframe that has incomplete values
merge_final[
    (merge_final["final_checker"] != 84)
    & (~merge_final.BUYER_STATE.isin(["FL", "LA", "MS", "SC"]))
]

,BUYER_STATE,BUYER_COUNTY,YEAR/MONTH,FULL_STATES,MME/CAP,checker,final_checker


In [420]:
# Create a column call "PolicyState"
merge_final["PolicyState"] = (
    (merge_final["BUYER_STATE"] == "FL")
    | ((merge_final["BUYER_STATE"] == "TX"))
    | (merge_final["BUYER_STATE"] == "WA")
)

In [421]:
# Create a column call Post
merge_final["Post"] = (
    (
        (
            (merge["BUYER_STATE"] == "FL")
            | (merge["BUYER_STATE"] == "LA")
            | (merge["BUYER_STATE"] == "MS")
            | (merge["BUYER_STATE"] == "SC")
        )
        & (merge["Year"] >= 2010)
    )
    | (
        (
            (merge["BUYER_STATE"] == "TX")
            | (merge["BUYER_STATE"] == "AR")
            | (merge["BUYER_STATE"] == "NM")
            | (merge["BUYER_STATE"] == "KS")
        )
        & (merge["Year"] >= 2007)
    )
    | (
        (
            (merge["BUYER_STATE"] == "WA")
            | (merge["BUYER_STATE"] == "OR")
            | (merge["BUYER_STATE"] == "CO")
            | (merge["BUYER_STATE"] == "CA")
        )
        & (merge["Year"] >= 2012)
    )
)

In [422]:
merge_final.head()

,BUYER_STATE,BUYER_COUNTY,YEAR/MONTH,FULL_STATES,MME/CAP,checker,final_checker,PolicyState,Post
0,FL,ALACHUA,2006,Florida,0.3449,7.0,7,True,False
1,FL,ALACHUA,2007,Florida,0.3925,7.0,7,True,False
2,FL,ALACHUA,2008,Florida,0.4683,7.0,7,True,False
3,FL,ALACHUA,2009,Florida,0.5728,7.0,7,True,False
4,FL,ALACHUA,2010,Florida,0.6101,7.0,7,True,True


In [425]:
# Convert to CSV
merge_final.to_csv(
    r"D:/Duke University (MIDS 2022)/Fall 2020/IDS 720/Team Project/SHIPMENT_merge_v2.csv",
    index=False,
    header=True,
)